# <h1><font color =#68bb59>Music Generation</font></h1>

### To use this Notebook:
Hit 'File' > Save a copy in Drive!</center><br>

For this project we will be using [Magenta](https://magenta.tensorflow.org/), a Python library created by Google that helps you generate art and music. We will specifically be using the music generation bits in `magenta.music` with all of its nice features that make it easy to:
- Make music using some of the neat abstractions and utilities in the library
- Use Machine Learning models to generate music

Rather than jump into highly theoretical knowledge, this project will be a good chance to explore applied ML, which focuses on having ML in your toolbox to apply to problems. If at the end of this project you are interested in learning more about ML (incuding the super hard theory) we have included references and resources that you can look into at the end of this notebook!

[More information about this project!](https://www.twilio.com/blog/generate-music-python-neural-networks-magenta-tensorflow)



## <h2><font color=#68bb59>Useful helpers</font></h2>
There are a lot of other helper methods sprinkled around the `magenta.music` codebase that you might need but not know where to find. Here are some of our favourites:

- [converting](https://github.com/tensorflow/magenta/blob/master/magenta/music/midi_io.py) between MIDI and NoteSequences
- [trimming, concatenating and expanding](https://github.com/tensorflow/magenta/blob/master/magenta/music/sequences_lib.py) NoteSequences
- [colab notebook](https://github.com/tensorflow/magenta/blob/master/magenta/music/notebook_utils.py) utils

# <h1><font color=#68bb59>Week One: Install libraries and dependencies!</font></h1>
If you're going to use `Magenta`, you will first need to install it and its dependencies. Some of the later examples will also download other dependencies (such as models and checkpoints). You might get some angry looking red "errors" but it's alright.

In [0]:
#@test {"output": "ignore"}

print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print 'Importing libraries and defining some helper functions...'
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print '🎉 Done!'
print magenta.__version__ 
print tensorflow.__version__

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 131331 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
     |████████████████████████████████| 1.5MB 5.1MB/s 
     |████████████████████████████████| 26.6MB 1.2MB/s 
     |████████████████████████████████| 2.7MB 38.7MB/s 
     |████████████████████████████████| 942kB 52.9MB/s 
     |████████████████████████████████| 2.3MB 38.2MB/s 
     |████████████████████████████████| 204kB 49.0MB/s 
     |█████████████████████████████

W0725 19:51:19.184289 140509047469952 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/pipelines/statistics.py:132: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0725 19:51:22.582952 140509047469952 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/note_sequence_io.py:60: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.



🎉 Done!
1.1.2
1.14.0


# <h1><font color=#68bb59>Week Two: Lets create data to train our models: Note Sequences!</font></h1>

Now that you have your environment all set up, our first real step will be to convert a collection of MIDI files into <font color=#68bb59><b>NoteSequences</b></font>. NoteSequences are [protocol buffers](https://developers.google.com/protocol-buffers/), which is a fast and efficient data format, and easier to work with than MIDI files. Assuming you've copied this notebook into your Google Drive, you should now copy our folder of MIDI files -- or [create your own](https://www.vgmusic.com) -- into the same folder that your Colab notebook is in.

## <h2><font color=#68bb59>A. Let's give our notebook access to our files</font></h2>

To use files (i.e. MIDI files) stored in your googleDrive, use this line to allow this notebook to have access to your google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# To list the files in your drive use !ls 
!ls "/content/gdrive/My Drive/"

ls: cannot access '/content/gdrive/Shared with me/': No such file or directory


In [0]:
# Load the file source path to your midiSource
midiSource =  "/content/gdrive/My Drive/[Codeology] Music Generation Project/MIDI"
midiSource = midiSource + "/AnimalCrossing1PM.mid"
print(midiSource)

/content/gdrive/My Drive/[Codeology] Music Generation Project/MIDI/AnimalCrossing1PM.mid


## <h2><font color=#68bb59>B. Now let's convert our MIDI file to a NoteSequence by using the Magenta Library</font></h2>

Everything in `Magenta` is centered around [NoteSequences](https://github.com/tensorflow/magenta/blob/master/magenta/protobuf/music.proto#L27), which is an abstract representation of a series of notes, each with different pitches, instruments and strike velocities, much like [MIDI](https://en.wikipedia.org/wiki/MIDI).

In [0]:
# Convert your MIDI file to a NoteSequence (try looking at docs / source code to figure out the method to do this!)
notes = mm.midi_file_to_note_sequence(midiSource)

W0725 17:49:11.511734 139851981547392 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/midi_io.py:192: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
# If your file was successfully converted you should see a note sequence graph and be able to play your tune
mm.plot_sequence(notes)
mm.play_sequence(notes,synth=mm.fluidsynth)

Loading BokehJS ...

## <h2><font color=#68bb59>C. (Optional) Let's try creating our own Note Sequence</font></h2>

Magenta also allows you to build Note Sequences from scratch! Check out these examples below, you can play around with them or generate your own based on these examples.

In [0]:
from magenta.protobuf import music_pb2


twinkle_twinkle = music_pb2.NoteSequence()

# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
mm.plot_sequence(twinkle_twinkle)

# This is a colab utility method that plays a NoteSequence.
mm.play_sequence(twinkle_twinkle,synth=mm.fluidsynth)

# Here's another NoteSequence!
teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=80)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=80)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=80)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=80)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

mm.plot_sequence(teapot)
mm.play_sequence(teapot,synth=mm.synthesize)

# You can also explore different instruments! Notice the instrument argument.
drums = music_pb2.NoteSequence()

drums.notes.add(pitch=36, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=38, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=46, start_time=0, end_time=0.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.25, end_time=0.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.375, end_time=0.5, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.5, end_time=0.625, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=50, start_time=0.5, end_time=0.625, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=36, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=38, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=45, start_time=0.75, end_time=0.875, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=36, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=46, start_time=1, end_time=1.125, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=42, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=48, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.notes.add(pitch=50, start_time=1.25, end_time=1.375, is_drum=True, instrument=10, velocity=80)
drums.total_time = 1.375

drums.tempos.add(qpm=60)

# This is a colab utility method that visualizes a NoteSequence.
mm.plot_sequence(drums)

# This is a colab utility method that plays a NoteSequence.
mm.play_sequence(drums,synth=mm.fluidsynth)

Congrats, you have finished week two!
Be sure to check out the [other jupyter noteboook](https://github.com/jennxiao/musicGenerationPrep/blob/master/Music%20Generation%20Week%202.ipynb) if you want more practice in Machine Learning topics!

# <h1><font color=#68bb59>Week Three: Lets start training our model</font></h1>

`magenta.music` has several Machine Learning models, each with different strengths. All models are built with [Tensorflow](https://www.tensorflow.org), so they will run faster if you can run them on a GPU. Here are some of the most popular ones:

- [**MelodyRNN**](https://github.com/tensorflow/magenta/tree/master/magenta/models/melody_rnn) - you give it a NoteSequence, and it continues it in the style of your original NoteSequence.
- [**MusicVAE**](https://github.com/tensorflow/magenta/tree/master/magenta/models/music_vae) - generates brand new NoteSequences or interpolates between two sequences.
- [**Onsets and Frames**](https://github.com/tensorflow/magenta/tree/master/magenta/models/onsets_frames_transcription) -- transcribes piano audio


Now that we know how to use `NoteSequences`, adding some basic Machine Learning is a continuation of that. The pattern for using any of these models is:
  * Load `magenta.music` (which we already know how to do)
  * Create a model from a downloaded checkpoint (i.e. where the weights, or the encoding, of the model lives)
  * Ask the model to do something.

## <h2><font color=#68bb59>Melody RNN</font></h2>

A MelodyRNN is an [LSTM-based](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) language model for musical notes -- it is best at continuing a NoteSequence that you give it. To use it, you will need to give it a sequence to continue and the model will generate and return the rest of the sequence.

This example shows how to use the basic Melody RNN model -- check out the [docs](https://github.com/tensorflow/magenta/tree/master/magenta/models/melody_rnn) for the other models, such as `lookback_rnn` and `attention_rnn`, which use other approaches.

## <h2><font color=#68bb59>A. Initialize the Model</font></h2>

We'll begin by downloading the model from the Magenta library.

In [0]:
print 'Downloading model bundle. This will take less than a minute...'
mm.notebook_utils.download_bundle('basic_rnn.mag', '/content/')
bundle = mm.sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

# Initialize the model.
print "Initializing Melody RNN..."
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print '🎉 Done!'

W0725 17:52:27.138616 139851981547392 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/notebook_utils.py:196: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



W0725 17:52:30.374741 139851981547392 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0725 17:52:30.407458 139851981547392 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:70: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0725 17:52:30.442497 139851981547392 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/model.py:71: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0725 17:52:30.525681 139851981547392 meta_graph.py:935] The saved meta_graph is possibly from an older release:
'mode

Initializing Melody RNN...


W0725 17:52:30.732438 139851981547392 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/magenta/music/sequence_generator.py:159: The name tf.gfile.DeleteRecursively is deprecated. Please use tf.io.gfile.rmtree instead.



🎉 Done!


## <h2><font color=#68bb59>B. Train the Model</font></h2>

As with any ML model, there will be certain settings you can adjust that might yield better results. With Melody RNN, you can configure <b><font color = #68bb59>the number of steps</font></b> the new sequence will be, as well as the <b><font color = #68bb59>temperature</font></b> of the result -- the higher the temperature, the more random (and less like the input) your sequence will be. You can play around with these values and see how the resulting sequences are different:

The file doesn't need to be monophonic, but it does need to have at least 1 track that is monophonic so our melody extractor can find something that looks like a melody. There's a bit more information about how this works here:

In [0]:
# Model options. Change these to get different generated sequences! 

input_sequence = notes  # change this to a Note Sequence Object like twinkle_twinkle or drums
num_steps = 1888 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth=mm.fluidsynth)

NameError: ignored

If for some reason you are not able to play your generated MIDI file -- do not fret! You can the code cell in the next section below to download your MIDI file manually and use whatever music player you have available on your computer to play the MIDI file.

## <h2><font color=#68bb59>Converting a `NoteSequence` to MIDI</font></h2>

When you called the `play_sequence` method above, this converted the `NoteSequence` to MIDI, and created an HTML widget to play it. This method is specially made for colab notebooks, so it won't work inside your Python script. That method uses the Magenta built-in [conversion methods](https://github.com/tensorflow/magenta/blob/master/magenta/music/midi_io.py#L51), which you can use in your python script:

In [0]:
# This creates a file called `drums_sample_output.mid`, containing the drums solo we've been using.
mm.sequence_proto_to_midi_file(drums, 'drums_sample_output.mid')

# This is a colab utility method to download that file. In your Python script, you 
# would just write it to disk.
files.download('drums_sample_output.mid')

NameError: ignored


## <h2><font color=#68bb59>C. (Optional) Train different types of RNN Models:</font></h2>

Magenta has multiple types of RNN Models available for use. 

<b><font color = #68bb59>Basic</font></b> 
* LSTM model that uses "basic one-hot encoding" to represent extracted melodies as input. For training, all examples are transposed to the MIDI pitch range [48, 84] and outputs will also be in this range. This is what we used!

<b><font color = #68bb59>Mono</font></b> 
* Similar to Basic but is able to use the full 128 MIDI pitches.

<b><font color = #68bb59>Lookback </font></b> 
* Introduces custom inputs and labels. The custom inputs allow the model to more easily recognize patterns that occur across 1 and 2 bars. They also help the model recognize patterns related to an events position within the measure. 
* You should hear melodies that have more musical structure / less randomness. This [blog post](https://magenta.tensorflow.org/2016/07/15/lookback-rnn-attention-rnn/)  has some sample outputs. You can also read through the events_to_input and events_to_label methods in `magenta/music/encoder_decoder.py` and `magenta/music/melody_encoder_decoder.py` to see how the custom inputs and labels are actually being encoded.

<b><font color = #68bb59>Attention </font></b>
* Allows the model to more easily access past information without having to store that information in the RNN cell's state. This allows the model to more easily learn longer term dependencies, and results in melodies that have longer arching themes. This [blog post](https://magenta.tensorflow.org/2016/07/15/lookback-rnn-attention-rnn/)  gives more information.

In [0]:
# Initialize a different model.
print "Initializing Melody RNN..."
generator_map = melody_rnn_sequence_generator.get_generator_map()
#choose one of 'basic_rnn', 'mono_rnn', lookback_rnn', or 'attention_rnn'
melody_rnn2.initialize()
melody_rnn2 = generator_map['mono_rnn'](checkpoint=None, bundle=bundle) 

Then you can just repeat the training steps the same as before!

In [0]:
# Model options. Change these to get different generated sequences! 

input_sequence = notes  # change this to a NoteSequence Object like twinkle_twinkle or drums
num_steps = 1888 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn2.generate(input_sequence, generator_options) #this is updated to new RNN model

mm.plot_sequence(sequence)
mm.play_sequence(sequence, synth=mm.fluidsynth)


# <h1><font color = #68bb59>CONGRATS</font></h1>
You made it to the end! I hope that you found this project insightful and that you were able to fall in love with machine learning! Below are some resources you can check out if you'd like to continue exploring the ML universe.

## <h2><font color=#68bb59>Resources</font></h2>
* UC Berkeley Machine Learning DeCal
* UC Berkeley CS 189
* etc.